In [1]:
from helper_functions import *

In [2]:
MERGED_PATH = "mergedfiles"
TRANSFORMED_PATH = "transformedfiles"
if not os.path.exists(TRANSFORMED_PATH):
    os.mkdir(TRANSFORMED_PATH)

In [3]:
files = get_remaining_files(MERGED_PATH)
files

['empresas.csv',
 'estabelecimentos.csv',
 'id_cnae_fiscal_principal.csv',
 'id_motivo_situacao_cadastral.csv',
 'id_natureza_juridica.csv',
 'id_qualificacoes.csv',
 'municipios.csv',
 'paises.csv',
 'simples.csv',
 'socios.csv']

# Empresas

In [4]:
empresas_path = os.path.join(MERGED_PATH, files[0])

In [5]:
empresas = pd.read_csv(empresas_path, chunksize=100_000, dtype={"cnpj_basico": "str"})

header = True
save_path = os.path.join(TRANSFORMED_PATH, files[0])
for chunk in empresas:

    chunk = chunk.loc[chunk["capital_social"].str.find("capital_social") == -1]
    chunk = chunk.fillna(value={"natureza_juridica": 0,
                                "qualificacoes": 0,
                                "porte_empresa": 0})

    # chunk["natureza_juridica"] = chunk["natureza_juridica"].fillna(0)
    # chunk["qualificacoes"] = chunk["qualificacoes"].fillna(0)
    # chunk["porte_empresa"] = chunk["porte_empresa"].fillna(0)
    
    chunk["capital_social"] = chunk["capital_social"].apply(lambda x: x.replace(",", ".")).astype(float)

    chunk.to_csv(save_path, header=header, index=False, mode="a")
    header = False

# Estabelecimentos

In [6]:
estabelecimentos_path = os.path.join(MERGED_PATH, files[1])

In [7]:
estabelecimentos = pd.read_csv(estabelecimentos_path, chunksize=100_000, dtype={"cnpj_basico": "str",
                                                                                "cnpj_ordem": "str",
                                                                                "cnpj_dv": "str",
                                                                                "data_situacao_cadastral": "str",
                                                                                "data_inicio_atividade": "str",
                                                                                "cep": "str",
                                                                                "ddd1": "str",
                                                                                "telefone1": "str",
                                                                                "ddd2": "str",
                                                                                "telefone2": "str",
                                                                                "ddd_fax": "str",
                                                                                "situacao_especial": "str",
                                                                                "data_situacao_especial": "str"})

header = True
save_path = os.path.join(TRANSFORMED_PATH, files[1])

for chunk in estabelecimentos:

    chunk = chunk.loc[chunk["cnpj_basico"].str.find("cnpj_basico") == -1]
    chunk = chunk.fillna(value={"identificador": 0,
                                "situacao_cadastral": 1,
                                "cod_pais": 999,
                                "cod_municipio": 0,
                                "cnae_fiscal_principal": 0,
                                "motivo_situacao_cadastral": 0})

    chunk.to_csv(save_path, header=header, index=False, mode="a")
    header = False

/tmp/ipykernel_15111/3885930778.py:18: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in estabelecimentos:
/tmp/ipykernel_15111/3885930778.py:18: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in estabelecimentos:
/tmp/ipykernel_15111/3885930778.py:18: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in estabelecimentos:
/tmp/ipykernel_15111/3885930778.py:18: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in estabelecimentos:
/tmp/ipykernel_15111/3885930778.py:18: DtypeWarning: Columns (8,26) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in estabelecimentos:
/tmp/ipykernel_15111/3885930778.py:18: DtypeWarning: Columns (8,26) have mixed types. Specify dtype option on import or set low_memor

KeyboardInterrupt: 